<a href="https://colab.research.google.com/github/zoya-ivanova/-Data-Science/blob/main/Sem_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Урок 11. Качество модели и её улучшение
Тебе предстоит разработать модель машинного обучения для предсказания вероятности оттока клиентов в телекоммуникационной компании.
У тебя есть набор данных, который содержит различные признаки о клиентах, такие как возраст, пол, тип подключения, длительность пользования услугами и т.д.

Твоя задача состоит в следующем:

1. Загрузить данные и провести предварительный анализ данных.

2. Подготовить данные для обучения модели. Это может включать удаление ненужных признаков, заполнение пропущенных значений и нормализацию данных.

3. Разделить данные на обучающую и тестовую выборки.

4. Выбрать модель машинного обучения, которую ты хочешь использовать для предсказания оттока клиентов. Например, можно использовать модель классификации, такую как логистическая регрессия или случайный лес.

5. Обучить модель на обучающей выборке и оценить ее производительность на тестовой выборке.

6. Провести анализ результатов и оценить важность различных признаков для предсказания оттока клиентов.

7. При необходимости, провести дополнительные эксперименты, чтобы улучшить производительность модели. Это может включать изменение параметров модели, добавление новых признаков или использование другой модели машинного обучения.

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

df = pd.read_csv("telecom_churn.csv")
df.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   State                   3333 non-null   object 
 1   Account length          3333 non-null   int64  
 2   Area code               3333 non-null   int64  
 3   International plan      3333 non-null   object 
 4   Voice mail plan         3333 non-null   object 
 5   Number vmail messages   3333 non-null   int64  
 6   Total day minutes       3333 non-null   float64
 7   Total day calls         3333 non-null   int64  
 8   Total day charge        3333 non-null   float64
 9   Total eve minutes       3333 non-null   float64
 10  Total eve calls         3333 non-null   int64  
 11  Total eve charge        3333 non-null   float64
 12  Total night minutes     3333 non-null   float64
 13  Total night calls       3333 non-null   int64  
 14  Total night charge      3333 non-null   

In [8]:
# Удаление колонки State
df = df.drop(columns=['State'])

Удаление колонки State: не несет полезной информации для анализа, она не влияет на целевую переменную (Churn). Удаление поможет уменьшить размер данных и улучшить производительность моделей.

In [9]:
# Кодирование колонок International plan и Voice mail plan
df['International plan'] = df['International plan'].map({'Yes': 1, 'No': 0})
df['Voice mail plan'] = df['Voice mail plan'].map({'Yes': 1, 'No': 0})

Столбцы International plan и Voice mail plan содержат категориальные данные (Yes и No). Для большинства алгоритмов машинного обучения требуется числовое представление данных. Для это заменим Yes на 1, а No на 0 (преобразуем в числовой формат).

In [10]:
# Перевод целевой переменной Churn в числа
df['Churn'] = df['Churn'].astype(int)

Целевая переменная Churn изначально представлена как булевое значение (True или False). Для удобства анализа и моделирования преобразуем True в 1, а False в 0, это позволит использовать стандартные методы машинного обучения, которые работают с числовыми данными.

In [11]:
# Создание матрицы объект-признак X и вектора целевой переменной y
X = df.drop(columns=['Churn'])
y = df['Churn']

# Проверка изменений
X.head()
y.head()

,Churn
0,0
1,0
2,0
3,0
4,0


In [12]:
# Проверка измений
df.head()

,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,128,415,0,1,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,107,415,0,1,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,137,415,0,0,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,84,408,1,0,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,75,415,1,0,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0


In [13]:
# Проверка баланса классов
class_counts = y.value_counts()
print("Баланс классов:")
print(class_counts)

Баланс классов:
Churn
0    2850
1     483
Name: count, dtype: int64


Данные несбалансированы: 2850 наблюдений для класса 0 (не ушли) и 483 наблюдения для класса 1 (ушли). Это может повлиять на производительность моделей машинного обучения, так как они могут быть склонны к предсказанию более частого класса.

In [14]:
# Разделение данных на тренировочные и тестовые
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Вывод размеров тренировочной и тестовой выборок
print("Размер тренировочной выборки:", X_train.shape)
print("Размер тестовой выборки:", X_test.shape)

Размер тренировочной выборки: (2333, 18)
Размер тестовой выборки: (1000, 18)


In [21]:
# Инициализация StandardScaler

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Обучение StandardScaler на тренировочных данных и применение к тренировочным данным
X_train_scaled = scaler.fit_transform(X_train)

# Применение StandardScaler к тестовым данным (transform, not fit_transform!)
X_test_scaled = scaler.transform(X_test)


In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score

In [23]:
# Обучение логистической регрессии
log_reg = LogisticRegression()
log_reg.fit(X_train_scaled, y_train)

# Обучение метода опорных векторов с линейным ядром
svc = SVC(kernel='linear', probability=True)
svc.fit(X_train_scaled, y_train)

# Предсказания на тестовых данных
y_pred_log_reg = log_reg.predict_proba(X_test_scaled)[:, 1] # Now X_test_scaled is defined
y_pred_svc = svc.predict_proba(X_test_scaled)[:, 1]

# Оценка качества моделей с помощью метрики ROC AUC
roc_auc_log_reg = roc_auc_score(y_test, y_pred_log_reg)
roc_auc_svc = roc_auc_score(y_test, y_pred_svc)

print(f"ROC AUC для логистической регрессии: {roc_auc_log_reg:.4f}")
print(f"ROC AUC для метода опорных векторов: {roc_auc_svc:.4f}")

ROC AUC для логистической регрессии: 0.8282
ROC AUC для метода опорных векторов: 0.7403


Модель логистической регрессии имеет более высокую метрику ROC AUC (0.8282) по сравнению с методом опорных векторов (0.7404). Это значит, что логистическая регрессия лучше справляется с задачей классификации в данном случае.

In [24]:
from sklearn.tree import DecisionTreeClassifier

# Обучение решающего дерева на немасштабированных данных
dt_ns = DecisionTreeClassifier()
dt_ns.fit(X_train, y_train)

DecisionTreeClassifier()

In [26]:
# Предсказания на тестовых данных
y_pred_ns = dt_ns.predict_proba(X_test)[:, 1]

# Оценка качества модели с помощью метрики ROC AUC
roc_auc_ns = roc_auc_score(y_test, y_pred_ns)
print(f"ROC AUC для решающего дерева: {roc_auc_ns:.4f}")

ROC AUC для решающего дерева: 0.8368


Для решающих деревьев масштабирование признаков не является критически важным шагом, так как они не зависят от расстояний между точками данных. Без масштабирования модель показывает 0,84, это хорошая способность различать классы.

Логистическая регрессия: ROC AUC 0.8282

Метод опорных векторов (SVM): ROC AUC 0.7404

Решающее дерево: ROC AUC 0.84

Таким образом, наилучший результат с метрикой ROC AUC 0.84 показало дерево решений, дерево лучше справилось с задачей классификации в данном случае.

In [ ]:
# Инициализация и обучение модели RandomForestRegressor
model_rf = RandomForestRegressor()
model_rf.fit(X_train_scaled, y_train)

RandomForestRegressor()

In [ ]:
# Прогнозирование
y_pred = model_rf.predict(X_test_scaled)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# Установка порога (например, 0.5)
threshold = 0.5

# Преобразование прогнозов в дискретные классы
y_pred_binary = np.where(y_pred > threshold, 1, 0)

# Оценка производительности модели
accuracy = accuracy_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")

Accuracy: 0.957
Precision: 0.9464285714285714
Recall: 0.7412587412587412
F1-score: 0.8313725490196079


Accuracy (Точность): 0.957 доля правильных предсказаний от общего числа предсказаний, модель правильно классифицировала 95.7% всех примеров. Подходит если классы сбалансированы (у нас - нет).

Precision (Точность): 0.946 доля правильных положительных предсказаний от всех положительных предсказаний. Из всех примеров, которые модель предсказала как положительные, 94.6% действительно были положительными. Важна для диагностики заболеваний.

**F1-score (F1-мера)**: **0.831** - гармоническое среднее между точностью и полнотой. F1-мера учитывает как ложные положительные, так и ложные отрицательные предсказания.

Recall (Полнота): 0.74, когда критично не пропустить положительные примеры. Например, при обнаружении мошенничества, где важно выявить как можно больше случаев мошенничества, даже если это приведет к некоторым ложным срабатываниям

###Случайный лес

показал хорошие результаты, особенно с точки зрения точности (Accuracy) и точности (Precision).

Его метрики показывают, что случайный лес хорошо справляется с задачей классификации, особенно когда важно минимизировать ложные положительные предсказания (высокая точность).


###Нам важно предсказать,

будет ли клиент пользоваться услугами или откажется от них, метрика Precision (Точность) становится особенно важной, потому что необходимо минимизировать количество ложных положительных предсказаний, то есть случаев, когда модель ошибочно предсказывает, что пользователь будет пользоваться услугами, хотя на самом деле он откажется.


###**Сравнение с решающим деревом:**
Решающее дерево показало лучший результат по метрике ROC AUC (0.84), что указывает на его лучшую способность различать классы. Если необходимо сбалансировать точность и способность различать классы, решающее дерево является хорошим вариантом.

###**Вывод:**
**Хотя решающее дерево показало лучшую общую производительность по метрике ROC AUC. Предпочтительным выбором для нашей задачи классификации в данном случае будет Случайный лес, поскольку его точность (Precision 0.946) выше, и нам важно минимизировать ложные положительные предсказания**